In [17]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='light')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


Trend line of what yLUNA is being used for?  PRISM Farm, yLUNA Staking, LPing, or Nothing.

In [18]:
prism_addr = 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw'
yluna_addr = 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz'
pluna_addr = 'terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2'
pLuna_PRISM_Pair = 'terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs5vs'
yLuna_PRISM_Pair = 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'

In [19]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [20]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [32]:
class PrismBalanceDataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.transfers = '9651b671-1f85-4133-8384-20042dc1adb1'
        self.boost_activation = 'ccbaf92f-6aaa-4460-a65f-83814431fd17'
        self.prism_forge = '2a364c52-9c37-4540-81c7-e5b890f0ef6b'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
    
    def load(self):
        self.load_from_url()
        self.load_from_csv()
        for df in [self.transfers_df, self.boost_activation_df, self.prism_forge_df,
                  self.transfers_df_from_csv, self.boost_activation_df_from_csv, self.prism_forge_df_from_csv]:
            df.columns = [c.lower() for c in df.columns]
       
        self.transfers_df = self.transfers_df.append(self.transfers_df_from_csv).drop_duplicates()
        self.boost_activation_df = self.boost_activation_df.append(self.boost_activation_df_from_csv).drop_duplicates()
        self.prism_forge_df = self.boost_activation_df.append(self.prism_forge_df_from_csv).drop_duplicates()
            
    def load_from_url(self):
        self.transfers_df = self.claim(self.transfers)
        self.boost_activation_df = self.claim(self.boost_activation)
        self.prism_forge_df = self.claim(self.prism_forge)
        
    def write_to_csv(self):
        self.transfers_df.to_csv(f'{self.path_to_data}/transfers.csv')
        self.boost_activation_df.to_csv(f'{self.path_to_data}/boost_activation.csv')
        self.prism_forge_df.to_csv(f'{self.path_to_data}/prism_forge.csv')
        
    def load_from_csv(self):
        self.transfers_df_from_csv = pd.read_csv(f'{self.path_to_data}/transfers.csv', index_col=0)
        self.boost_activation_df_from_csv = pd.read_csv(f'{self.path_to_data}/boost_activation.csv', index_col=0)
        self.prism_forge_df_from_csv = pd.read_csv(f'{self.path_to_data}/prism_forge.csv', index_col=0)
        
        
    def parse_transfers(self):
        transfers_df = self.transfers_df
        transfers_df.block_timestamp=transfers_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        transfers_df.block_timestamp=transfers_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        transfers_df.block_timestamp=transfers_df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        transfers_df['hr'] = transfers_df.block_timestamp.str[:-5] + '00:00.000'
        transfers_df['day'] = transfers_df.block_timestamp.str[:-9]
        self.transfers_df= transfers_df
        send_tx = transfers_df[['block_timestamp','from_','amount','day']]
        send_tx.columns = ['block_timestamp','user','amount','day']
        send_tx['amount_signed'] = -send_tx.amount
        receive_tx = transfers_df[['block_timestamp','to_','amount','day']]
        receive_tx.columns = ['block_timestamp','user','amount','day']
        receive_tx['amount_signed'] = receive_tx.amount
        self.all_transfers = send_tx.append(receive_tx)
        
    def parse(self):
        self.parse_transfers()
        


In [33]:
prism_dp = PrismBalanceDataProvider(claim, get_url, '../data/prism/')
prism_dp.load()
prism_dp.parse()